In [ ]:
import numpy as np

### Classe para o problema

In [ ]:
class SlidingPuzzle:
  def __init__(self, num_blocos):
    self.num_blocos = num_blocos

  def _encontra_posicao(self, estado, elemento):
    for i in range(self.num_blocos):
      for j in range(self.num_blocos):
        if elemento == estado[i, j]:
          return i, j
    return None, None

  def verifica_estados(self, atual, objetivo):
    for i in range(self.num_blocos):
      for j in range(self.num_blocos):
        if atual[i, j] != objetivo[i, j]:
          return False
    return True

  def expande_estados(self, atual):
    novos_estados = []

    linha, coluna = self._encontra_posicao(atual, 0)

    # Cima
    if linha > 0:
      nova_linha = linha - 1
      novo_estado = np.copy(atual)

      bloco_alvo = atual[nova_linha, coluna]
      novo_estado[nova_linha, coluna] = 0
      novo_estado[linha, coluna] = bloco_alvo
      novos_estados.append(novo_estado)

    # Baixo
    if linha < self.num_blocos - 1:
      nova_linha = linha + 1
      novo_estado = np.copy(atual)

      bloco_alvo = atual[nova_linha, coluna]
      novo_estado[nova_linha, coluna] = 0
      novo_estado[linha, coluna] = bloco_alvo
      novos_estados.append(novo_estado)

    # Esquerda
    if coluna > 0:
      nova_coluna = coluna - 1
      novo_estado = np.copy(atual)

      bloco_alvo = atual[linha, nova_coluna]
      novo_estado[linha, nova_coluna] = 0
      novo_estado[linha, coluna] = bloco_alvo
      novos_estados.append(novo_estado)

    # Direita
    if coluna < self.num_blocos - 1:
      nova_coluna = coluna + 1
      novo_estado = np.copy(atual)

      bloco_alvo = atual[linha, nova_coluna]
      novo_estado[linha, nova_coluna] = 0
      novo_estado[linha, coluna] = bloco_alvo
      novos_estados.append(novo_estado)

    return novos_estados

  def distancia_manhattan(self, atual, objetivo):
    total_dist = 0

    for i in range(self.num_blocos):
      for j in range(self.num_blocos):
        elemento = atual[i, j]
        linha, coluna = self._encontra_posicao(objetivo, elemento)

        dist = abs(linha - i) + abs(coluna - j)
        total_dist += dist

    return total_dist

### Classe para a busca

In [ ]:
import heapq

class BuscaGulosa():
  def __init__(self, problema):
    self.problema = problema

  def _verifica_visitado(self, estado, estados_visitados):
    for i in estados_visitados:
      if self.problema.verifica_estados(i, estado):
        return True
    return False

  def busca(self, inicio, fim):
    p_fila = []
    
    # H, ID, elemento
    heapq.heappush(p_fila, (0, 0, inicio))

    id_estado = 0
    solucao_encontrada = False
    estados_visitados = []
    cont_estados = 0

    while len(p_fila) != 0:
      atual = heapq.heappop(p_fila)[2]
      estados_visitados.append(atual)

      if self.problema.verifica_estados(atual, fim):
        solucao_encontrada = True
        break
      else:
        cont_estados += 1
        print("Visitando #", cont_estados)
        novos_estados = self.problema.expande_estados(atual)

        for i in novos_estados:
          if not self._verifica_visitado(i, estados_visitados):
            id_estado += 1
            prioridade = self.problema.distancia_manhattan(i, fim)

            print(i, "\nh = ", prioridade)
            heapq.heappush(p_fila, (prioridade, id_estado, i))

    return solucao_encontrada, estados_visitados, cont_estados  

### Main

In [ ]:
start = np.matrix([[2, 8, 3], [1, 6, 4], [7, 0, 5]])
target = np.matrix([[1, 2, 3], [8, 0, 4], [7, 6, 5]])

In [ ]:
problema = SlidingPuzzle(3)
gulosa = BuscaGulosa(problema)

In [ ]:
solucao, visitados, passos = gulosa.busca(start, target)

if solucao:
  print("Solução encontrada com", passos)
else:
  print("Solução não foi encontrada")